# Spatial SQL functions workshop exercise

## Assessing wildfire exposure for commercial property risks

<img src="assets/wildfire.png">



### Installation of dependencies

In [0]:
%pip install -r requirements.txt
%restart_python

In [0]:
from datetime import datetime, timedelta
import geopandas as gpd
import math
import shapely

from lonboard import Map, PolygonLayer, ScatterplotLayer

import pyspark.sql
import pyspark.sql.functions as F
from pyspark.sql.window import Window

## Configuration
Replace these values with your own

In [0]:
user_email = spark.sql("SELECT CURRENT_USER() AS u").first()["u"]
user_name = user_email.split("@")[0].replace(".", "_")
CATALOG = user_name
SCHEMA = "geoday"

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")

In [0]:
spark.conf.set("spark.databricks.geo.st.enabled", "true")

## Overview of available datasets

### Overture Maps Buildings
+ **Description**: Building footprint dataset. Contains building footprint polygons with some basic characteristics of the building where available.
+ **Scope**: Global
+ **Provider**: [Overture Maps Foundation](https://overturemaps.org/) (via [CARTO](https://carto.com/))
+ **Marketplace listing**: [⧉](https://marketplace.databricks.com/details/ccacdfa3-b85d-4065-bd70-efa673c197e1/CARTO_Overture-Maps-Buildings)

### Overture Maps Places
+ **Description**: Addresses and locations (as point geometries) of points-of-interest including landmarks, businesses, transport hubs etc. with their names, functions and similar attributes.
+ **Scope**: Global
+ **Provider**: [Overture Maps Foundation](https://overturemaps.org/) (via [CARTO](https://carto.com/))
+ **Marketplace listing**: [⧉](https://marketplace.databricks.com/details/aca03de7-a670-4e3f-8ee8-b0f319f0a247/CARTO_Overture-Maps-Places)

### Wildfire Risk USA (sample)
+ **Description**: Wildfire hazard and risk assessment database. Identifies uniquely generated FireShed polygons covering the United States with corresponding risk scores. Selection of models used to compute risks depending population density (Wildland / Intermix / Interface).
+ **Scope**: USA (Marketplace sample is limited to Marin County, CA)
+ **Provider**: [Precisely](https://www.precisely.com/)
+ **Marketplace listing**: [⧉](https://marketplace.databricks.com/details/7ef74135-019f-4ec4-9c91-caf9b50233af/Precisely_Wildfire-Risk-USA-Sample)

### ECMWF 15-day operational weather forecast
+ **Description**: Detailed weather forecasts that are produced by the Integrated Forecasting System (IFS) (i.e. the 'physical' weather model) maintained by the [European Centre for Medium-Range Weather Forecasts (ECMWF)](https://www.ecmwf.int/). These operational forecasts are the main high-resolution deterministic forecast produced by ECMWF and extend out to the next 15-days ahead, initially in 3-hour timesteps (for the first five days), moving to 6-hour steps thereafter. They are published as [GRIB messages](https://confluence.ecmwf.int/display/CKB/What+are+GRIB+files+and+how+can+I+read+them#WhatareGRIBfilesandhowcanIreadthem-HowtoreadGRIBfiles) for a large number of weather variables for points in a regular 0.25° grid.
+ **Scope**: Global
+ **Provider**: [ECMWF]()
+ **Marketplace listing**: N/A (will be downloaded) [terms of use](https://apps.ecmwf.int/datasets/licences/general/)

## Getting started

First task: get access to the above Marketplace datasets.
1) Follow the link to each asset's Marketplace listing;
2) Click "Get Access" and login with your Databricks credentials; and
3) Follow the instructions to deploy the asset into a new Catalog in your Unity Catalog metastore.

When complete, run the following cell to check these have been correctly deployed.

In [0]:
def catalog_exists(catalog_name):
  try:
    spark.sql(f"SHOW CATALOGS LIKE '{catalog_name}'")
    return True
  except:
    return False

assert all(catalog_exists(c) for c in [
  "precisely_wildfire_risk_usa_sample",
  "carto_overture_maps_places",
  "carto_overture_maps_buildings"
])

## The task
In this task, we aim to build a very simplistic model of wildfire risk for all of the commercial property in Marin County, CA.

Though simplistic, this model will account for the frequency and severity of historical fires, the physical distance between the property and a nearby fire-station and, as an optional extension, the outlook for weather in the region.

In order to achieve this, we will need to perform a number of subtasks:

### Sub-task (1): identify points-of-interest (commercial property risks)
+ Extract the building footprints for all _commercial_ properties in the **Overture Maps Buildings** dataset that fall within the zone covered by the **Wildfire Risk USA** data.

+ This will require aggregating (unioning) the FireShed zones in the table `precisely_wildfire_risk_usa_sample.riskdata.wfr_usa_fireriskpro` into one large polygon and computing the bounding box of said polygon.

+ The parameters of this bounding box can then be used to filter the **Overture Maps Buildings** dataset stored at `carto_overture_maps_buildings.carto.building`.
  + **Hint** This table contains a number of fields prefixed with `__carto_` that describe the bounding box of each building footprint.

+ Use the provided code to plot a selection of these on a map using the `lonboard` package.

In [0]:
%sql
SELECT * FROM precisely_wildfire_risk_usa_sample.riskdata.wfr_usa_fireriskpro

In [0]:
fireriskpro_tref = "precisely_wildfire_risk_usa_sample.riskdata.wfr_usa_fireriskpro"

xmin, xmax, ymin, ymax = (
  spark.table(fireriskpro_tref)
  .select(F.expr("st_aswkb(st_union_agg(WKT))").alias("geom"))
  .select(
    F.expr("st_xmin(geom)").alias("xmin"),
    F.expr("st_xmax(geom)").alias("xmax"),
    F.expr("st_ymin(geom)").alias("ymin"),
    F.expr("st_ymax(geom)").alias("ymax"),
  )
).first()

print(
  "Fire risk pro dataset bounds:\n"
  f"{xmin=}, {xmax=}, {ymin=}, {ymax=}"
)

In [0]:
buildings_tref = f"{CATALOG}.{SCHEMA}.buildings"
spark.sql(f"DROP TABLE IF EXISTS {buildings_tref}")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {buildings_tref} AS
SELECT
  id,
  geometry,
  class,
  subtype,
  height,
  variant_get(names, '$.primary', 'string') as building_name
FROM
  carto_overture_maps_buildings.carto.building
WHERE
  class = 'commercial'
  AND __carto_xmin >= {xmin}
  AND __carto_xmax <= {xmax}
  AND __carto_ymin >= {ymin}
  AND __carto_ymax <= {ymax}""")

In [0]:
buildings_df = spark.table(buildings_tref)
building_cols = buildings_df.columns
print(f"{buildings_df.count()=}")

In [0]:
display(buildings_df)

In [0]:
buildings_pdf = buildings_df.toPandas()
buildings_pdf["geometry"] = gpd.GeoSeries.from_wkb(
  buildings_pdf.geometry, crs="EPSG:4326"
  )
buildings_gdf = gpd.GeoDataFrame(buildings_pdf, geometry="geometry")

m = Map([PolygonLayer.from_geopandas(
    buildings_gdf, get_fill_color=[255, 0, 0]
)])

displayHTML(m.to_html())

### Sub-task (2): identify fire station locations in the same area

+ Now, query the **Overture Maps Places** dataset to find the locations of local fire stations.
  + In order to extract e.g. primary usage category from the VariantType field `categories`, you'll need to employ the Spark expression `variant_get` like so:
  `variant_get(categories, '$.primary', 'string')`
  + Since we want to compute the distance from each property in our analysis 'set', you might want to 'buffer' the bounding box before applying it to the **Places** data.

+ As before, you can use the provided code to plot these locations on a map.

In [0]:
firestations_tref = f"{CATALOG}.{SCHEMA}.firestations"

spark.sql(f"DROP TABLE IF EXISTS {firestations_tref}")

In [0]:
buffer = 0.05 # buffer around area of interest in degrees (~5km)

spark.sql(f"""
CREATE OR REPLACE TABLE {firestations_tref} AS
WITH ft AS
(
  SELECT
    id,
    geometry,
    variant_get(names, '$.primary', 'string') AS firestation_name,
    variant_get(categories, '$.primary', 'string') AS firestation_primary_category,
    variant_get(addresses, '$.list[0].element', 'map<string, string>') AS address_map
  FROM
    carto_overture_maps_places.carto.place
  WHERE
    variant_get(categories, '$.primary', 'string') = 'fire_department'
    AND __carto_xmin >= {xmin} - {buffer}
    AND __carto_xmax <= {xmax} + {buffer}
    AND __carto_ymin >= {ymin} - {buffer}
    AND __carto_ymax <= {ymax} + {buffer}
)
SELECT
  ft.id as firestation_building_id,
  ft.geometry as firestation_geometry,
  ft.firestation_name,
  ft.firestation_primary_category,
  named_struct(
    'freeform', ft.address_map['freeform'],
    'locality', ft.address_map['locality'],
    'region', ft.address_map['region'],
    'postcode', ft.address_map['postcode'],
    'country', ft.address_map['country']
    ) as address
  FROM ft
  """)

In [0]:
firestations_df = spark.table(firestations_tref)
firestation_cols = firestations_df.columns
print(f"{firestations_df.count()=}")

In [0]:
display(firestations_df)

In [0]:
firestations_pdf = firestations_df.toPandas()
firestations_pdf["geom"] = gpd.GeoSeries.from_wkb(
  firestations_pdf.firestation_geometry, crs="EPSG:4326"
  )
firestations_gdf = gpd.GeoDataFrame(firestations_pdf, geometry="geom")

point_layer = ScatterplotLayer.from_geopandas(
    firestations_gdf, radius_scale=100, get_fill_color=[0, 255, 0],
    get_line_color=[0, 0, 0], stroked=True
)
m = Map([point_layer])

displayHTML(m.to_html())

### Sub-task (3): compute the "as-the-crow-flies" distance from each property to its nearest fire station

+ Since our dataset is small, we can do this naively i.e. compute the distance (in metres if possible) between every property and every fire station.
  + **Hint** The `st_distance` function returns results in the same units as its inputs, e.g. degrees for geometries expressed in geographic coordinates and metres for those in cartesian coordinates.
  + In order to transform from geographic to cartesian coordinates, we can use the `st_transform` method. You'll need to find a suitable projected CRS to transform them into, e.g. `Universal Transverse Mercator Zone 10 North` [⧉](https://spatialreference.org/ref/epsg/32610/).

+ We can then filter the results set to only keep the closest option of those computed. Applying a window function that uses `row_number()` is a usually a good approach here.
  + **Hint** Get the assistant to help you define the window specification if you get stuck.

+ Pick a fire station and map the properties that are in its 'catchment', i.e. for whom this is their nearest option.

In [0]:
def add_utm10n_geom(df: pyspark.sql.DataFrame, col: str) -> pyspark.sql.DataFrame:
    newcol = f"{col}_utm10N"
    op = F.expr(f"st_aswkb(st_transform({col}, 32610))")
    return df.withColumn(newcol, op)

In [0]:
buildings_projected_tref = f"{CATALOG}.{SCHEMA}.buildings_projected"

(
    buildings_df
    .transform(add_utm10n_geom, "geometry")
    .write.saveAsTable(buildings_projected_tref, mode="overwrite")
)

buildings_projected_df = spark.table(buildings_projected_tref)

In [0]:
# all_firestations_tref = f"{CATALOG}.{SCHEMA}.all_firestations"

distance_to_all_firestations = (
    buildings_projected_df
    .alias("b")
    .crossJoin(
        firestations_df.transform(add_utm10n_geom, "firestation_geometry").alias("f")
        )
    .withColumn("firestation_distance", F.expr("st_distance(geometry_utm10N, firestation_geometry_utm10N)"))
    .selectExpr("* EXCEPT (geometry_utm10N, firestation_geometry_utm10N)")
    # .write
    # .saveAsTable(all_firestations_tref, mode="overwrite")
)

# distance_to_all_firestations = spark.table(all_firestations_tref)

display(distance_to_all_firestations)

In [0]:
ws_distance = Window.partitionBy("id").orderBy("firestation_distance")

closest_firestation = (
  distance_to_all_firestations
  .withColumn("rank", F.row_number().over(ws_distance))
  .where(F.col("rank") == 1)
  .drop("rank")
)

display(closest_firestation)

In [0]:
def visualize_firestation(name: str):
    filtered_pdf = (
        closest_firestation.where(F.col("firestation_name") == name)
        .select(*building_cols, "firestation_distance")
        .toPandas()
    )

    filtered_pdf["geometry"] = gpd.GeoSeries.from_wkb(filtered_pdf["geometry"])
    gdf = gpd.GeoDataFrame(filtered_pdf, geometry="geometry", crs="EPSG:4326")

    # Filter for the firestation name
    firestation_pdf = (
        firestations_df.select(*firestation_cols)
        .where(F.col("firestation_name") == name)
        .toPandas()
    )

    firestation_pdf["firestation_geometry"] = gpd.GeoSeries.from_wkb(firestation_pdf["firestation_geometry"])
    gdf_point = gpd.GeoDataFrame(firestation_pdf, geometry="firestation_geometry", crs="EPSG:4326")

    m = Map(
        [
            ScatterplotLayer.from_geopandas(
                gdf_point, radius_scale=15, get_fill_color=[255, 0, 0]
            ),
            PolygonLayer.from_geopandas(
                gdf,
                get_fill_color=[0, 0, 255],
                extruded=True,
                get_elevation=gdf["height"].fillna(1),
            ),
        ]
    )
    m.set_view_state(pitch=59.0)
    displayHTML(m.to_html())

In [0]:
visualize_firestation("Southern Marin Fire Station No. 7")

### Sub-task (4): overlay the risk parameters for each property based on its fireshed location

You should now have a dataset that contains the building footprints of the insured risks and the name and distance of the nearest fire station. The core component of our simple model of wildfire risk will be the risk scores computed by **Precisely** as part of their **Fire Risk Pro** product.

In contrast to other, similar products which offer risk scores across the cells of a square grid, Precisely have divided up their product area into 'fire sheds', analagous to the watersheds commonly used in the field of hydrology.

From their documentation:
> Many wildfire-related datasets are delivered in a format that divides the landscape into square grids. A square has very little to do with how a fire burns and the variation from square to square can be difficult to interpret. Fire Risk Pro takes a different approach.

> All datasets Fire Risk Pro uses to create a final analysis (many of which come in “square” format) are integrated into polygons that reflect the landscape and how wildfires actually burn within that landscape. Fire Risk Pro divides the data into F and S fire sheds that are based on the topography (hills and valleys) of the landscape. These fire sheds tend to correlate to vegetation or wildland fuels and determine how a fire might burn and behave.

> This means that fire sheds are more likely to divide geography into landscape-based units that exhibit similarities in vegetation and slope, and resultant fire behavior. The wildland and intermix modules of Fire Risk Pro ... use this concept of fire sheds to aggregate the landscape. tend to correlate to vegetation or wildland fuels and determine how a fire might burn and behave.

In order to include fire shed level attributes into our per-property model, we shall have to ascertain the fire shed(s) in which a given property is located; a join between two polygon datasets referred to as an intersection join.

We have a spatial SQL function for performing exactly this kind of join `st_intersects`. Our preview documentation states:
> `st_intersects(geom1, geom2)`
> * **Input types:** (GEOMETRY,GEOMETRY) / (STRING, STRING) / (STRING, BINARY) / (BINARY, STRING) / (BINARY, BINARY)
> * **Return type:** BOOLEAN
> * **Description:** ST_Intersects takes as input two geometries and returns true iff the two geometries intersect.

So if we wish to, we can just use the function directly in our join as a join condition. For a dataset of this size, it will take a little time but should complete without issue. Try it and see!

In [0]:
firesheds_tref = f"precisely_wildfire_risk_usa_sample.riskdata.wfr_usa_fireriskpro"

fireshed_cols = ["NOHARM_ID", "NOHARMCLS", "NOHARMODEL", "RISK50", "SEVERITY", "FREQUENCY", "WKT"]

firesheds_df = (
  spark.table(firesheds_tref)
  .select(*fireshed_cols)
  )

In [0]:
# Naive join to obtain fireshed variables for each building

with_fs_risk = (
  closest_firestation.alias("b")
  .join(firesheds_df.alias("fs"), on=F.expr("st_intersects(b.geometry, fs.WKT)"))
  )

display(with_fs_risk.orderBy("RISK50", ascending=False))

+ How might we account for the scenario where the property straddles more than one fireshed?

  How could a filter be applied to your results in order to, for example, select the join match with the highest `RISK50` rating?
    + **Hint** We already did something very similar when selecting the 'nearest' fire station.

In [0]:
# Post-join filter
ws_risk = Window.partitionBy("id").orderBy(F.col("RISK50").desc())

buildings_with_unique_risk_rating = (
  with_fs_risk
  .withColumn("rank", F.rank().over(ws_risk))
  .where(F.col("rank") == 1)
  .drop("rank")
  )

display(buildings_with_unique_risk_rating)

In [0]:
def visualize_fireshed(results_df: pyspark.sql.dataframe.DataFrame, id: str):

    filtered_pdf = (
      results_df
      .where(F.col("NOHARM_ID") == id)
      .drop(*fireshed_cols)
      .toPandas()
      )
    
    firestation = filtered_pdf["firestation_building_id"][0]
    firestation_geom_wkb = (
      firestations_df
      .where(F.col("firestation_building_id") == firestation)
      ).first()["firestation_geometry"]
    firestation_geom = shapely.wkb.loads(bytes(firestation_geom_wkb))
    
    filtered_pdf['geometry'] = gpd.GeoSeries.from_wkb(filtered_pdf['geometry'])
    gdf = gpd.GeoDataFrame(filtered_pdf, geometry='geometry', crs='EPSG:4326')

    fireshed_pdf = (
      results_df
      .where(F.col("NOHARM_ID") == id)
      .limit(1)
      .select(*fireshed_cols)
      .toPandas()
      )

    fireshed_pdf['geometry'] = gpd.GeoSeries.from_wkt(fireshed_pdf['WKT'])
    gdf_fs = gpd.GeoDataFrame(fireshed_pdf, geometry='geometry', crs='EPSG:4326')

    m = Map([
      PolygonLayer.from_geopandas(gdf_fs, get_fill_color=[255, 0, 0], opacity=0.3),
    ], show_tooltip=True)
    m.add_layer(
      PolygonLayer.from_geopandas(gdf, get_fill_color=[0, 0, 255]),
      )
    m.add_layer(
      ScatterplotLayer.from_geopandas(
        gpd.GeoDataFrame.from_records([[firestation_geom]])
        .set_geometry(0)
        .set_crs('EPSG:4326'),
        radius_scale=50,
        get_fill_color=[0, 255, 0]
        ),
    )
    displayHTML(m.to_html())

In [0]:
visualize_fireshed(buildings_with_unique_risk_rating, 'CP1286009320')

In [0]:
visualize_fireshed(buildings_with_unique_risk_rating, 'CX1302766484')

### 4a: efficiency and scalability
That same documentation also describes how you might look to employ a spatial grid indexing system in order to speed up the join and scale to much larger regions for your analysis.

> Spatial predicates are more compute intensive. For performance purposes, we strongly urge customers to spatially index their data using productized functions `h3_lonlatash3` and `h3_tessellateaswkb`.

In case you're not familiar with H3, you can read more about it [here](https://h3geo.org/). The tl;dr explanation would be along the lines of: "geohashing with hexagons".

In our case, because both sides of the join contain polygon geometries, we should use `h3_tessellateaswkb` to first decompose the geometries into multiple smaller, simpler 'chips' (the pink polygons in the picture below) that are intersections of the original geometry (red filled area) H3 grid indexing system (black hexagons).

<img src="assets/h3-tessellation.png" width="450px" title="blah">

*Decomposition of a Precisely fire shed at an H3 resolution of 9.*

The `h3_tessellateaswkb` function is documented thus:
> `h3_tessellateaswkb(geom, resolution, [return_core_chips])`
> * **Input types:** (STRING, INT[, BOOLEAN])/(GEOGRAPHY, INT[, BOOLEAN])/(BINARY, INT[, BOOLEAN])
> * **Return type:** STRUCT{BIGINT, BOOLEAN, BINARY}
> * **Description:** Tessellate input vector data using h3 global grid indexing.
>
>    Takes as input two required and one optional argument:
>   * A geography object (either a GEOGRAPHY value, or a STRING/BINARY value representing a geography in GeoJSON, WKB, or WKT format).
>   * A resolution value (value between 0 and 15, inclusive).
>   * [Optional] A boolean value indicating whether the WKB description of core H3 cells should be present in the output, or replaced by NULL values (see below).
>
>   The function returns an array of named structs. The array represents a tessellation of the input geography using H3 cells at the specified resolution. The tessellation is essentially a decomposition of the geography using a minimal covering set of H3 cells.
>
>   The structs have three fields: "cellid", "core", "chip".
>   * The "cellid" is an H3 cell at the specified resolution that is a member of the minimal covering set of H3 cells of the geography at the specified resolution.
>   * The "core" field is a boolean value that indicates whether the H3 cell of the struct is fully contained in the geography (for core cells the intersection of the geography with the cell's polygon is the cell's polygon).
>   * The "chip" field is a BINARY value, and corresponds to the intersection of the input geography with the cell's polygon, represented in WKB format. If the case where the cell is fully contained inside the geography and the third optional argument is set to false, the "chip" field value would be NULL instead.

Once we decompose the geometries on both our left and right sides of the join, we can then perform the join as an equi-join (exact matches between keys) between H3 indexes, before (if necessary) filtering out false-positives using `st_intersects` between the property geometry and the fire shed chip, e.g.

```sql
SELECT
...
FROM properties p
  INNER JOIN indexed_firesheds f
  ON p.h3 = f.h3
WHERE (
  f.core 
  OR st_intersects(p.geometry, f.chip)
  )
```

Go ahead and try it!

In [0]:
firesheds_indexed = (
  firesheds_df
  .withColumn("chip", F.expr("explode(h3_tessellateaswkb(WKT, 9))"))
  .select(*fireshed_cols, "chip.*")
  )

In [0]:
with_fs_risk_h3 = (
  closest_firestation
  .withColumn("h3", F.expr("explode(h3_tessellateaswkb(geometry, 9))"))
  .select(*building_cols, *firestation_cols, "h3.*")
  .alias("b")
  .join(firesheds_indexed.alias("fs"), on="cellid")
  .where("st_geometrytype(fs.chip) = 'ST_Polygon'")
  .where(
    F.col("fs.core") |
    F.expr("st_intersects(b.geometry, fs.chip)")
    )
  .select(*building_cols, *firestation_cols, *fireshed_cols)
)

In [0]:
ws_risk = Window.partitionBy("id").orderBy(F.col("RISK50").desc())

buildings_with_unique_risk_rating_h3 = (
  with_fs_risk_h3
  .distinct()
  .withColumn("rank", F.rank().over(ws_risk))
  .where(F.col("rank") == 1)
  .drop("rank")
)

display(with_fs_risk_h3.orderBy(F.col("RISK50").desc()))

In [0]:
visualize_fireshed(buildings_with_unique_risk_rating_h3, "CP1286009320")

In [0]:
visualize_fireshed(buildings_with_unique_risk_rating_h3, "CX1302766484")

### Sub-task (5): Add weather into the model

Take a look at the notebook `custom_spark_data_source_ifs` in the folder `./utils`.

Inside, there's an example implementation of a custom Pyspark DataSource that can be used to obtain weather forecast data from the ECMWF.

As mentioned in the introductory section of this notebook, this data is provided in 0.25° grids in GRIB format.

We could simply download this data from the ECMWF Open Data Portal, interrogate it using a tool such as [xarray](https://xarray.dev/) and extract the values to support exactly the kind of analysis we are intending to perform here.

This would quickly become difficult to manage if looking across multiple forecasts, variables and regions. For that type and scale of application, we can take an xarray based workflow and wrap it inside a Spark DataSource reader using the new [pyspark.sql.datasource.DataSource](https://spark.apache.org/docs/4.0.0-preview2/api/python/reference/pyspark.sql/api/pyspark.sql.datasource.DataSource.html) class.

A more thorough examination of the topic is offered [here](https://docs.databricks.com/aws/en/pyspark/datasources) but we can use this implementation to obtain forecast data that can be joined to our dataset as follows:

#### Reading ECMWF forecasts using Spark and storing the results into UC
Reading GRIB files with xarray requires a native code dependency installed through the `cfgrib` python package. We use this command to check that it has been correctly installed.

In [0]:
%sh python -m cfgrib selfcheck

In [0]:
%run ./utils/custom_spark_data_source_ifs

In [0]:
spark.dataSource.register(IFSDataSource)

#### Data filtering
##### Spatial filtering
Sadly, we can't speed up the download process by filtering with a spatial boundary. Instead, we will read all values and drop any out-of-bounds rows before writing into Delta.
##### Time horizon
By default, we will download the forecast produced yesterday, since that is guaranteed to be available at any point during the day. Current day forecasts from the 0h UTC run can take up to 9 hours to be published to the open data store.

In [0]:
forecast_date = (
  datetime.now() - timedelta(days=1) # D - 1
  ).strftime('%Y%m%d')

In [0]:
weather_df = (
  spark.read.format("ifs")
  .options(**{
    "forecastDate": forecast_date,
    # download the midnight model run
    "forecastTime": 0,
    # only process subset of forecast steps (forecast horizon in hours)
    #  adjust to your needs (more hours, more spark tasks to be completed)
    "forecastHorizon": "3",
    # extract temperature, 'u' and 'v' components of windspeed and total rainfall
    "variables": "t2m,u100,v100,tp"
  })
  .load()
  )


In [0]:
display(weather_df)

In [0]:
weather_data_tref = f"{CATALOG}.{SCHEMA}.weather_forecast_raw"
print(f'Saving ifs data to: {weather_data_tref}')

(
  weather_df
  .where(F.col("x").between(math.floor(xmin), math.ceil(xmax)))
  .where(F.col("y").between(math.floor(ymin), math.ceil(ymax)))
  .write.saveAsTable(weather_data_tref, mode="overwrite")
)

forecast_df = spark.table(weather_data_tref)

In [0]:
display(forecast_df.where("variable='t2m'"))

Databricks visualization. Run in Databricks to view.

From this point, it is completely up to you to decide how include this data in your analysis. 

Here are some things you may want to consider:
+ How will you join this data to your current dataset? Do you need to perform a point-in-polyon join using `st_contains`? Or is it good enough to extract the lons and lats of each building and round these to the nearest quarter degree?
+ Will the data need to be aggregated after joining? If, for example, you choose to align the data to fire sheds rather than properties, then it's quite possible you will have more than one data point for each fire shed.
+ What is the relative importance of temperature vs. wind speed? Does the direction of the wind matter, if so, how will you account for this?

In [0]:
with_weather = ( ... )

In [0]:
dbutils.notebook.exit("0")

In [0]:
fireshed_pdf = (
      firesheds_df
      .where(F.col("NOHARM_ID") == "CP1286009320")
      .toPandas()
      )

fireshed_indexed_pdf = (
      firesheds_indexed
      .where(F.col("NOHARM_ID") == "CP1286009320")
      .withColumn("h3_geom", F.expr("h3_boundaryaswkb(cellid)"))
      .toPandas()
      )

chips_pdf = fireshed_indexed_pdf.copy(deep=True)

fireshed_pdf['fs_geometry'] = gpd.GeoSeries.from_wkt(fireshed_pdf['WKT'])
fireshed_indexed_pdf['h3_geometry'] = gpd.GeoSeries.from_wkb(fireshed_indexed_pdf['h3_geom'])
chips_pdf['chip_geometry'] = gpd.GeoSeries.from_wkb(chips_pdf['chip'])

gdf_fs = gpd.GeoDataFrame(fireshed_pdf, geometry='fs_geometry', crs='EPSG:4326')
gdf_h3 = gpd.GeoDataFrame(fireshed_indexed_pdf, geometry='h3_geometry', crs='EPSG:4326')
gdf_chip = gpd.GeoDataFrame(chips_pdf, geometry='chip_geometry', crs='EPSG:4326')

m = Map([
  PolygonLayer.from_geopandas(gdf_fs, get_fill_color=[255, 0, 0], opacity=0.3),
  PolygonLayer.from_geopandas(gdf_h3, filled=False, get_line_width=5),
  PolygonLayer.from_geopandas(gdf_chip, filled=False, get_line_color=[255, 0, 255], get_line_width=10),
])
displayHTML(m.to_html())
